In [1]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

# Montando o Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Caminho dos arquivos no Google Drive
vendas_etanol_consolidado_path = '/content/drive/MyDrive/Dados TCC/vendas_etanol_consolidado.csv'
etanol_unificado_path = '/content/drive/MyDrive/Dados TCC/ProducaoCapacidade/Etanol_DadosAbertos_CSV_Unificado.csv'

# Ajustando a leitura dos datasets com o delimitador correto
vendas_etanol_consolidado = pd.read_csv(vendas_etanol_consolidado_path, delimiter=',')
vendas_etanol_consolidado.columns = ['mes', 'vendas', 'produto']

etanol_unificado = pd.read_csv(etanol_unificado_path, delimiter=',')
etanol_unificado.columns = ['mes_ano', 'regiao', 'estado', 'safra', 'producao_etanol_hidratado', 'producao_etanol_anidro', 'capacidade_etanol_anidro', 'capacidade_etanol_hidratado']

# Convertendo os campos de data para DateTime
vendas_etanol_consolidado['mes'] = pd.to_datetime(vendas_etanol_consolidado['mes'])
etanol_unificado['mes_ano'] = pd.to_datetime(etanol_unificado['mes_ano'])

# Extraindo os anos presentes em cada dataset
anos_vendas = vendas_etanol_consolidado['mes'].dt.year.unique()
anos_etanol = etanol_unificado['mes_ano'].dt.year.unique()

# Identificando anos que estão presentes em ambos os datasets
anos_comuns = set(anos_vendas).intersection(set(anos_etanol))

# Filtrando os dados para incluir apenas os anos comuns
vendas_etanol_consolidado_filtrado = vendas_etanol_consolidado[vendas_etanol_consolidado['mes'].dt.year.isin(anos_comuns)]
etanol_unificado_filtrado = etanol_unificado[etanol_unificado['mes_ano'].dt.year.isin(anos_comuns)]

# Unificando os datasets
# Primeiro, ajustando as colunas para garantir a união correta
vendas_etanol_consolidado_filtrado = vendas_etanol_consolidado_filtrado.rename(columns={'mes': 'mes_ano'})

# Removendo colunas 'estado' e 'regiao' do etanol_unificado_filtrado
etanol_unificado_filtrado = etanol_unificado_filtrado.drop(columns=['regiao', 'estado'])

# Realizando a união dos datasets com base na coluna 'mes_ano'
dataset_unificado = pd.merge(etanol_unificado_filtrado, vendas_etanol_consolidado_filtrado, on='mes_ano', how='inner')

# Função para interpolar valores faltantes ou zeros usando interpolação cúbica
def interpolate_cubic(df, columns):
    for column in columns:
        x = np.arange(len(df))
        y = df[column].replace(0, np.nan)
        mask = ~np.isnan(y)
        interp = interp1d(x[mask], y[mask], kind='cubic', fill_value="extrapolate")
        df[column] = interp(x)
    return df

# Interpolando os valores
colunas_quantidade = ['producao_etanol_hidratado', 'producao_etanol_anidro', 'capacidade_etanol_anidro', 'capacidade_etanol_hidratado', 'vendas']
dataset_unificado = interpolate_cubic(dataset_unificado, colunas_quantidade)

# Garantindo que as colunas de quantidade são floats com 2 casas decimais
dataset_unificado[colunas_quantidade] = dataset_unificado[colunas_quantidade].astype(float).round(2)

# Mostrando as primeiras 20 linhas do dataset unificado
print(dataset_unificado.head(20))

# Salvando o dataset unificado em um arquivo CSV
dataset_unificado.to_csv('/content/drive/My Drive/Dados TCC/dataset_unificado_interpolado.csv', index=False)


Mounted at /content/drive
      mes_ano  safra  producao_etanol_hidratado  producao_etanol_anidro  \
0  2019-01-01      0                     4524.0                31848.00   
1  2019-01-01      0                     4524.0                31848.00   
2  2019-01-01      0                    18564.0               175502.27   
3  2019-01-01      0                    18564.0               175502.27   
4  2019-01-01      0                  1671663.0               433251.00   
5  2019-01-01      0                  1671663.0               433251.00   
6  2019-02-01      0                      876.0                 3144.00   
7  2019-02-01      0                      876.0                 3144.00   
8  2019-02-01      0                   850388.0               386078.00   
9  2019-02-01      0                   850388.0               386078.00   
10 2019-03-01      0                   712198.0                93704.00   
11 2019-03-01      0                   712198.0                93704.00   